In [64]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import MemorySaver

In [65]:
load_dotenv()
llm = ChatOpenAI()

In [66]:
class jokeState(TypedDict):
    topic: str 
    joke: str 
    explanation: str 

In [67]:
def generate_joke(state: jokeState):
    prompt = f'generate a joke on the topic {state["topic"]}'
    response = llm.invoke(prompt).content

    return {'joke': response}

In [68]:
def generate_explanation(state: jokeState):
    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = llm.invoke(prompt).content

    return {'explanation': response}

In [69]:
state = StateGraph(jokeState)

state.add_node('generate_joke', generate_joke)
state.add_node('generate_explanation', generate_explanation)

state.add_edge(START, 'generate_joke')
state.add_edge('generate_joke', 'generate_explanation')
state.add_edge('generate_explanation', END)

checkpointer = MemorySaver()

workflow = state.compile(checkpointer=checkpointer)

config1 = {"configurable": {"thread_id": "1"}}

In [70]:
initial_state = {
    'topic': "Create a joke about monkey"
}

workflow.invoke(initial_state, config= config1)

{'topic': 'Create a joke about monkey',
 'joke': 'Why did the monkey like the banana? \n\nBecause it had a-peel!',
 'explanation': 'This joke is a play on words because "a peel" sounds like "appeal," which means something that is attractive or interesting to someone. In this case, the joke is saying that the monkey liked the banana because it had an "appeal" or "a peel," which is the outer skin of the banana. It\'s a light-hearted and punny way to explain why the monkey enjoyed eating the banana.'}

In [71]:
workflow.get_state(config1)
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Create a joke about monkey', 'joke': 'Why did the monkey like the banana? \n\nBecause it had a-peel!', 'explanation': 'This joke is a play on words because "a peel" sounds like "appeal," which means something that is attractive or interesting to someone. In this case, the joke is saying that the monkey liked the banana because it had an "appeal" or "a peel," which is the outer skin of the banana. It\'s a light-hearted and punny way to explain why the monkey enjoyed eating the banana.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f115309-7cdf-63d2-8002-47559b577d18'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2026-03-01T05:36:27.466607+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f115309-7071-6c80-8001-eed9b79c3679'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'Create a joke about monkey', 'joke': 'Why did the mo

In [72]:
workflow.get_state({"configurable": {"thread_id": "1","checkpoint_id":"1f115309-5d83-60d8-8000-4d9e526f877b"}})

StateSnapshot(values={'topic': 'Create a joke about monkey'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f115309-5d83-60d8-8000-4d9e526f877b'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2026-03-01T05:36:24.178290+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f115309-5d80-6590-bfff-6eec5cb0f7d7'}}, tasks=(PregelTask(id='420dd6ec-3687-75ef-da03-4c631f8dd568', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the monkey like the banana? \n\nBecause it had a-peel!'}),), interrupts=())

In [75]:
workflow.get_state(None, {"configurable": {"thread_id": "1","checkpoint_id":"1f115309-5d83-60d8-8000-4d9e526f877b"}})

TypeError: Pregel.get_state() takes 2 positional arguments but 3 were given